In [36]:
import math 
import sys
#a and b are coefficients for the gap function
def global_affine(a,b):
    dict_subst = {"a":0, "c": 1, "g":2, "t":3}
    
    for i in range(0, m+1):
        for j in range(0, n+1):
        # Compute D[i,j]
            v1,v2 = sys.maxsize,sys.maxsize
            if i>0 and j>=0 : 
                v1 = S[i-1,j]+(a+b)
            if i>1 and j>=0:
                v2 = D[i-1,j]+a
            D[i,j] = min(v1,v2)
                    
        # Compute I[i,j]
            v1,v2 = sys.maxsize,sys.maxsize
            if i>=0 and j>0:
                v1 = S[i,j-1]+(a+b)
            if i>=0 and j>1:
                v2 = I[i,j-1]+a
            I[i,j] = min(v1,v2)
                 
        # Compute S[i,j] 
            v1,v2,v3,v4 = sys.maxsize,sys.maxsize,sys.maxsize,sys.maxsize
            if i==0 and j==0 :
                v1 = 0                
            if i>0 and j>0:
                v2 = S[i-1,j-1] + subst_mat[dict_subst[seq1[i-1]], dict_subst[seq2[j-1]]]
            if i>0 and j>=0:
                v3 = D[i,j]
            if i>=0 and j>0:
                v4 = I[i,j]
            S[i,j] = min(v1,v2,v3,v4)        
    return S[i,j]

In [37]:
#a and b are the parameters for the affine gap function g(k) = a+b*k
def backtrack_affine(a,b):
    i = m
    j = n
    align1 = ""
    align2 = ""
    dict_subst = {"a":0, "c": 1, "g":2, "t":3}
    while (i>0 or j>0):
        if (i>0 and j>0) and (S[i,j] == S[i-1,j-1] + subst_mat[dict_subst[seq1[i-1]], dict_subst[seq2[j-1]]]):
            align1 = seq1[i-1] + align1
            align2 = seq2[j-1] + align2
            i = i-1
            j = j-1
        else:
            k = 1
            while True:
                if i>=k and S[i,j] == S[i-k,j] + (a+b*k):
                    l = i
                    while(l>=i-k+1):
                        align1 = seq1[l-1] + align1
                        align2 = "-" + align2
                        l = l-1
                    i = i-k
                    break
                elif j>=k and S[i,j] == S[i,j-k] + (a+b*k):
                    l = j
                    while(l>=i-k+1):
                        align1 = "-" + align1
                        align2 = seq2[l-1] + align2
                        l = l-1
                    j = j-k
                    break
                else:
                    k = k+1
    return align1, align2

In [38]:
seq1 = "aataat"
seq2 = "aagg"

m = len(seq1)
n = len(seq2)

S = np.full([m+1,n+1], None)
I = np.full([m+1,n+1], None)
D = np.full([m+1,n+1], None)

print(global_affine(5,5))
print(S)
print(backtrack_affine(5,5))

22
[[0 10 15 20 25]
 [10 0 10 15 20]
 [15 10 0 10 15]
 [20 15 10 5 15]
 [25 20 15 12 7]
 [30 25 20 17 14]
 [35 30 25 25 22]]
('aataat', 'a--agg')


In [39]:
import Bio

In [40]:
from Bio import SeqIO
import numpy as np

def read_FASTA(filename):
    records_dict = {}
    for seq_record in SeqIO.parse(filename, "fasta"):
        records_dict[seq_record.id] = seq_record.seq        
    return records_dict

In [65]:
#this reader assumes phylip-like format in txt file
# and that the matrix is DNA related and the nucleotides are in the order A,C,G,T
#therefore "skips" the first line and first character of each line following the first one
def read_subst_mtrx(filename):
    subst_mat = np.zeros((4,4))
    #skipping the first line
    f = open(filename,'r')
    f.readline()
    for i in range(0,4):
        line = f.readline()
        nums_in_line = line.split()
        for j in range(1,4):
            subst_mat[i,j-1] = nums_in_line[j]
    f.close()
    return subst_mat   
    

In [66]:
seq1 = list(read_FASTA("C:/Users/alzbe/Documents/AU_Bioinfo_Masters/Spring_2021/AiB/Projects/Project_01/seq1.fasta").values())[0].lower()
seq2 = list(read_FASTA("C:/Users/alzbe/Documents/AU_Bioinfo_Masters/Spring_2021/AiB/Projects/Project_01/seq2.fasta").values())[0].lower()
print(seq1)
print(seq2)
subst_mat = read_subst_mtrx("C:/Users/alzbe/Documents/AU_Bioinfo_Masters/Spring_2021/AiB/Projects/Project_02/subst_matrix.txt")
print(subst_mat)

[[0. 5. 2. 0.]
 [5. 0. 5. 0.]
 [2. 5. 0. 0.]
 [5. 2. 5. 0.]]


In [35]:
m = len(seq1)
n = len(seq2)

S = np.full([m+1,n+1], None)
I = np.full([m+1,n+1], None)
D = np.full([m+1,n+1], None)

print(global_affine(5,5))
print(S)
print(backtrack_affine(5,5))

395
[[0 10 15 ... 985 990 995]
 [10 0 10 ... 980 985 990]
 [15 10 0 ... 975 980 985]
 ...
 [985 980 975 ... 395 405 410]
 [990 985 980 ... 405 395 405]
 [995 990 985 ... 410 405 395]]
('ggcctaaaggcgccggtctttcgtaccccaaaatctcggcattttaagataagtgagtgttgcgttacactagcgatctaccgcgtcttatacttaagcgtatgcccagatctgactaatcgtgcccccggattagacgggcttgatgggaaagaacagctcgtc------tgtttacgtataaacagaatcgcctgggttcgc', 'gggctaaaggttagggtctttcacactaaagagtggt-gcgtatcgtggctaatgtaccgcttctggtatc-gtggcttacggc--cagacctacaagtactagacctga--gaactaatcttgtcgagccttccattgagggtaatgggagagaacatcgagtcagaagttattcttgtttacgtagaatcgcctgggtccgc')
